In [ ]:
#!pip install pyRDF2vec
#!pip install nest-asyncio
#!pip install rdflib
#!pip install owlready2
#!pip install aiohttp

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.manifold import TSNE
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.graphs import KG
from pyrdf2vec.walkers import RandomWalker
from owlready2 import *
import csv
from math import sqrt, pow, exp
from numpy import dot
from numpy.linalg import norm
import gensim
from gensim.matutils import cossim

In [ ]:
#Nouvelle approche
# Charger le fichier OWL
onto = get_ontology("/content/d3fend.owl").load()
off=[]
def takesubclassof(namefile,parentclass):
    # Ouvrir un fichier TSV pour écrire les données
    with open(namefile, mode='w') as file:
        writer = csv.writer(file, delimiter='\t')
        # Ecrire l'en-tête de colonne
        writer.writerow(["ID", "Nom"])

        # Compteur pour assigner des ID uniques
        id_counter = 1

        # Récupérer la classe parent
        parent_class = onto.search_one(iri = "*"+parentclass)

        # Parcourir toutes les sous-classes de la classe parent
        for sub_class in parent_class.subclasses():
            parent_class = onto.search_one(iri = "*"+sub_class.name)
            for p_class in parent_class.subclasses():
                # Écrire les données dans le fichier TSV
                if p_class not in off:
                    writer.writerow([id_counter, "http://d3fend.mitre.org/ontologies/d3fend.owl#"+p_class.name])
                    id_counter += 1
                    off.append(p_class)

In [ ]:
#Nouvelle approche
parent_class_nameexec = "OffensiveTechnique"
namefileexec='/content/offensive.tsv'
takesubclassof(namefileexec,parent_class_nameexec)

In [ ]:
#Nouvelle approche
data = pd.read_csv("/content/offensive.tsv", sep="\t")
entitiesdef = [entity for entity in data["Nom"]]

In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/d3fend.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def searchtech(techn):
    technique=[]
    sparql = """select ?a where{
                    <http://d3fend.mitre.org/ontologies/d3fend.owl#"""+techn+"""> <http://www.w3.org/2000/01/rdf-schema#label> ?a.
                }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['a'].toPython())
        technique.append(s)
    return technique

In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/d3fend.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def searchimpact(techn):
    technique=[]
    sparql = """select ?a where{
                    <http://d3fend.mitre.org/ontologies/d3fend.owl#"""+techn+"""> <http://www.w3.org/2000/01/rdf-schema#label> ?a.
                    <http://d3fend.mitre.org/ontologies/d3fend.owl#"""+techn+"""> rdfs:subClassOf+ <http://d3fend.mitre.org/ontologies/d3fend.owl#ImpactTechnique> .
                }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['a'].toPython())
        technique.append(s)
    return technique

In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/d3fend.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def searchpriv(techn):
    technique=[]
    sparql = """select ?a where{
                    <http://d3fend.mitre.org/ontologies/d3fend.owl#"""+techn+"""> <http://www.w3.org/2000/01/rdf-schema#label> ?a.
                    <http://d3fend.mitre.org/ontologies/d3fend.owl#"""+techn+"""> rdfs:subClassOf+ <http://d3fend.mitre.org/ontologies/d3fend.owl#PrivilegeEscalationTechnique> .
                }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['a'].toPython())
        technique.append(s)
    return technique

In [ ]:
impacttechnique=[]
imptechnique=[]
for entity in entitiesdef:
  tech=str(entity.split('#')[1])
  imptechnique=searchimpact(tech)
  privtechnique=searchpriv(tech)
  if len(imptechnique)!=0:
    impacttechnique.append(imptechnique)
  if len(privtechnique)!=0:
    impacttechnique.append(privtechnique)
for imp in impacttechnique:
  imptechnique.append(imp[0])

In [ ]:
techniquedef=[]
for entity in entitiesdef:
  tech=str(entity.split('#')[1])
  technique=searchtech(tech)
  techniquedef.append(technique)

In [ ]:
#Nouvelle approche
from owlready2 import *

vdo = get_ontology("/content/rdfxmlgraph.owl").load()

class_name = "Vulnerability"
# Récupération de la classe
#my_class = vdo.search_one(iri = "*" + class_name)
cveid="CVE-2017-0144"
# Récupération des individus de la classe
entities = vdo.Vulnerability.instances()
# Ouverture d'un fichier TSV pour écrire les résultats
with open("/content/vdo.tsv", "w") as f:
    f.write("ID\tvulnerability\n")
    """for i, entity in enumerate(entities):
        f.write("{}\t{}\n".format(i+1, "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#"+entity.name))"""
    f.write("{}\t{}\n".format(1, "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVE"+cveid))

In [ ]:
#Nouvelle approche
datavdo = pd.read_csv("/content/vdo.tsv", sep="\t")
entitiesvdo = [entity for entity in datavdo["vulnerability"]]

In [ ]:
#Nouvelle approche
import numpy as np
from numpy.linalg import norm
def cos_similarity(A,B):
    """ return cosine similarity between two lists """

    # compute cosine similarity
    cosine = np.dot(A,B)/(norm(A)*norm(B))
    return cosine

In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/d3fend.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def searchartifact():
    countermeasure=[]
    sparql = """select ?a where{
                    ?ar rdfs:subClassOf+ <http://d3fend.mitre.org/ontologies/d3fend.owl#Artifact> .
                    ?ar <http://www.w3.org/2000/01/rdf-schema#label> ?a .
                }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['a'].toPython())
        countermeasure.append(s)
    return countermeasure
artifact=searchartifact()

In [ ]:
corpus=[]
corpusart=[]
for item in artifact:
    split_words=item.split()
    corpus.append(split_words)
    corpusart.append(split_words)

In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/rdfxmlgraph.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def searchcontext():
    countermeasure=[]
    sparql = """select ?c where{
                    ?co rdf:type <http://www.semanticweb.org/anonymous/ontologies/2021/7/vdo#Context> .
                    ?co <http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#value> ?c .
                }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['c'].toPython())
        countermeasure.append(s)
    return countermeasure
context=searchcontext()

In [ ]:
for item in context:
    split_words=item.split()
    corpus.append(split_words)
    corpusart.append(split_words)

In [ ]:
offensivetech=[]
"""offens=[]
for off in techniquedef:
  if off[0].find('/')!=-1:
    offensivetech.append(off[0].split('/')[0])
    offensivetech.append(off[0].split('/')[1])
  else:
    offensivetech.append(off[0])
  offens.append(off[0])
print(offens)"""
for off in techniquedef:
  offensivetech.append(off[0])

In [ ]:


import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

def remove_prepositions(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpusoff=[]
for item in offensivetech:
  if item not in imptechnique:
    split_words=item.split()
    corpus.append(split_words)
    corpusoff.append(split_words)

In [ ]:
corpusimp=[]
for item in imptechnique:
    split_words=item.split()
    corpus.append(split_words)
    corpusimp.append(split_words)

In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/rdfxmlgraph.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def searchimpacts():
    countermeasure=[]
    sparql = """select ?i where{
                    ?im rdf:type <http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#LogicalImpact> .
                    ?im <http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#value> ?i .
                }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['i'].toPython())
        countermeasure.append(s)
    return countermeasure
logicalimpact=searchimpacts()

In [ ]:
for item in logicalimpact:
    split_words=item.split()
    corpus.append(split_words)
    corpusimp.append(split_words)

In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/rdfxmlgraph.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def searchmethod():
    countermeasure=[]
    sparql = """select ?m where{
                    ?me rdf:type <http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#ImpactMethod> .
                    ?me <http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#value> ?m .
                }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['m'].toPython())
        countermeasure.append(s)
    return countermeasure
impactmethod=searchmethod()

In [ ]:
for item in impactmethod:
    split_words=item.split()
    corpus.append(split_words)
    corpusoff.append(split_words)

In [ ]:
#from gensim.models import Word2Vec

# Entraînement du modèle word2vec
model = gensim.models.Word2Vec(corpusart, vector_size=200, window=5, min_count=1, workers=4, epochs=30)

# Enregistrement du modèle
model.save("word2vec.model")

In [ ]:
#from gensim.models import Word2Vec

# Entraînement du modèle word2vec
modelart = gensim.models.Word2Vec(corpusart, vector_size=200, window=5, min_count=1, workers=4, epochs=30)

# Enregistrement du modèle
modelart.save("word2vecart.model")

In [ ]:
from gensim.models import Phrases

# Train a bigram detector.
bigram_transformer = Phrases(corpusart)
# Apply the trained MWE detector to a corpus, using the result to train a Word2vec model.
modela = gensim.models.Word2Vec(bigram_transformer[corpusart], min_count=1)
# Enregistrement du modèle
modela.save("word2veca.model")

In [ ]:
modeloff = gensim.models.Word2Vec(corpusoff, vector_size=250, window=5, min_count=1, workers=10, epochs=50)

modeloff.save("word2vecoff.model")

In [ ]:
modelimp = gensim.models.Word2Vec(corpusimp, vector_size=250, window=5, min_count=1, workers=10, epochs=150)

modelimp.save("word2vecimp.model")

In [ ]:
 # Matching process starts here
import rdflib
from gensim.test.utils import common_texts
transformer = RDF2VecTransformer(walkers=[RandomWalker(4, 1, with_reverse=False, n_jobs=2)],
                                 verbose=2,
                                 embedder=Word2Vec(sentences=corpus))
embeddingsdef, literalsdef = transformer.fit_transform(
    KG(
        "/content/d3fend.owl",
       literals=[
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#executes",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#invokes",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#may-modify",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#modifies",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#accesses",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#may-access",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#runs",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#adds",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#may-add",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#produces",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#creates",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#may-create",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#uses",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#copies",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#may-transfer",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#reads",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#loads",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#may-invoke",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#queries",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#interprets",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#installs",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#hides",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#injects",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#forges",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#connects",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
            [
                "http://d3fend.mitre.org/ontologies/d3fend.owl#unmounts",
                "http://www.w3.org/2000/01/rdf-schema#label"
            ],
        ],
    ),
    entitiesdef
)

KG(location='/content/d3fend.owl', skip_predicates=set(), literals=[['http://d3fend.mitre.org/ontologies/d3fend.owl#executes', 'http://www.w3.org/2000/01/rdf-schema#label'], ['http://d3fend.mitre.org/ontologies/d3fend.owl#invokes', 'http://www.w3.org/2000/01/rdf-schema#label'], ['http://d3fend.mitre.org/ontologies/d3fend.owl#may-modify', 'http://www.w3.org/2000/01/rdf-schema#label'], ['http://d3fend.mitre.org/ontologies/d3fend.owl#modifies', 'http://www.w3.org/2000/01/rdf-schema#label'], ['http://d3fend.mitre.org/ontologies/d3fend.owl#accesses', 'http://www.w3.org/2000/01/rdf-schema#label'], ['http://d3fend.mitre.org/ontologies/d3fend.owl#may-access', 'http://www.w3.org/2000/01/rdf-schema#label'], ['http://d3fend.mitre.org/ontologies/d3fend.owl#runs', 'http://www.w3.org/2000/01/rdf-schema#label'], ['http://d3fend.mitre.org/ontologies/d3fend.owl#adds', 'http://www.w3.org/2000/01/rdf-schema#label'], ['http://d3fend.mitre.org/ontologies/d3fend.owl#may-add', 'http://www.w3.org/2000/01/rdf-

100%|██████████| 320/320 [00:00<00:00, 853.17it/s]


Extracted 320 walks for 320 entities (0.5109s)
Word2Vec(kwargs={'min_count': 0, 'sentences': [['Digital', 'Artifact'], ['Digital', 'Information', 'Bearer'], ['Resource'], ['Configuration', 'Resource'], ['Access', 'Control', 'Configuration'], ['Access', 'Control', 'Group'], ['User', 'Group'], ['Host', 'Group'], ['Access', 'Control', 'List'], ['Group', 'Policy'], ['Application', 'Configuration'], ['Application', 'Configuration', 'Database', 'Record'], ['Application', 'Process', 'Configuration'], ['Application', 'Rule'], ['Email', 'Rule'], ['Process', 'Environment', 'Variable'], ['Configuration', 'Database'], ['Application', 'Configuration', 'Database'], ['Shim', 'Database'], ['Configuration', 'Management', 'Database'], ['Configuration', 'Database', 'Record'], ['System', 'Configuration', 'Database', 'Record'], ['System', 'Configuration', 'Init', 'Database', 'Record'], ['Windows', 'Registry', 'Key'], ['Windows', 'Registry', 'Value'], ['Cloud', 'Configuration'], ['Cloud', 'Instance', 'Metad

100%|██████████| 320/320 [00:00<00:00, 3980.14it/s]


Extracted 320 literals for 320 entities (0.0871s)


In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/rdfxmlgraph.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def askcveexist(val):
    countermeasure=[]
    sparql = """SELECT ?i
              WHERE{
                    ?i <http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasIdentity> <http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#"""+val+"""> .
                    ?im rdf:type <http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#VulnerabilityIdentifier> .

                }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['i'].toPython())
        countermeasure.append(s)
    return countermeasure

In [ ]:
askres=askcveexist(cveid)

In [ ]:
import rdflib
from gensim.test.utils import common_texts


transformer = RDF2VecTransformer(walkers=[RandomWalker(4, 1, with_reverse=False, n_jobs=2)],
                                verbose=2,
                                embedder=Word2Vec(sentences=corpus))
embeddingsvdo, literalsvdo = transformer.fit_transform(
    KG(
        "/content/rdfxmlgraph.owl",
        #skip_predicates={"www.w3.org/1999/02/22-rdf-syntax-ns#type"},
      literals=[
            [
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasScenario",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasAction",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasImpactMethod",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#value"
            ],
            [
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasScenario",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasAction",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#resultsInImpact",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasLogicalImpact",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#value"
            ],
            [
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasScenario",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasAction",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#affectsContext",
                "http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#value"
            ],
        ],
    ),
    entitiesvdo,
)

KG(location='/content/rdfxmlgraph.owl', skip_predicates=set(), literals=[['http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasScenario', 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasAction', 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasImpactMethod', 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#value'], ['http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasScenario', 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasAction', 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#resultsInImpact', 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasLogicalImpact', 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#value'], ['http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#hasScenario', 'http://www.semanticweb.o

100%|██████████| 1/1 [00:00<00:00, 213.45it/s]


Extracted 1 walks for 1 entities (0.0847s)
Word2Vec(kwargs={'min_count': 0, 'sentences': [['Digital', 'Artifact'], ['Digital', 'Information', 'Bearer'], ['Resource'], ['Configuration', 'Resource'], ['Access', 'Control', 'Configuration'], ['Access', 'Control', 'Group'], ['User', 'Group'], ['Host', 'Group'], ['Access', 'Control', 'List'], ['Group', 'Policy'], ['Application', 'Configuration'], ['Application', 'Configuration', 'Database', 'Record'], ['Application', 'Process', 'Configuration'], ['Application', 'Rule'], ['Email', 'Rule'], ['Process', 'Environment', 'Variable'], ['Configuration', 'Database'], ['Application', 'Configuration', 'Database'], ['Shim', 'Database'], ['Configuration', 'Management', 'Database'], ['Configuration', 'Database', 'Record'], ['System', 'Configuration', 'Database', 'Record'], ['System', 'Configuration', 'Init', 'Database', 'Record'], ['Windows', 'Registry', 'Key'], ['Windows', 'Registry', 'Value'], ['Cloud', 'Configuration'], ['Cloud', 'Instance', 'Metadata'

100%|██████████| 1/1 [00:00<00:00, 831.87it/s]

Extracted 1 literals for 1 entities (0.0089s)


In [ ]:
print(literalsvdo)

[['Code Execution', ('Shutdown', 'Manipulation', 'Discovery'), 'Application']]


In [ ]:
listevul=[]
for lit in range(len(literalsvdo)):
  if type(literalsvdo[lit][1]) is tuple:
    for imp in literalsvdo[lit][1]:
      if type(literalsvdo[lit][0]) is tuple:
        for met in literalsvdo[lit][0]:
          litvdo=[entitiesvdo[lit],str(met),str(imp),literalsvdo[lit][2]]
          listevul.append(litvdo)
      else:
        litvdo=[entitiesvdo[lit],literalsvdo[lit][0],str(imp),literalsvdo[lit][2]]
        listevul.append(litvdo)
  else:
    if type(literalsvdo[lit][0]) is tuple:
      for met in literalsvdo[lit][0]:
        litvdo=[entitiesvdo[lit],str(met),literalsvdo[lit][1],literalsvdo[lit][2]]
        listevul.append(litvdo)
    else:
        litvdo=[entitiesvdo[lit],literalsvdo[lit][0],literalsvdo[lit][1],literalsvdo[lit][2]]
        listevul.append(litvdo)

In [ ]:
import heapq
listresult=[]
listresult2=[]
modeloff = gensim.models.Word2Vec.load("word2vecoff.model")
modelimp = gensim.models.Word2Vec.load("word2vecimp.model")
for vul in range(len(listevul)):
  sentences_similarity = np.zeros(len(offensivetech))
  sentences_similarity2 = np.zeros(len(offensivetech))
  for idx, sentence in enumerate(offensivetech):
    method=listevul[vul][1]
    impact=listevul[vul][2]
    sentence_words = sentence.split()
    for word in sentence_words:
      sim_to_sentence = 0
      sim_to_sentence2 = 0
      if type(impact)==str:
        logimpact=impact
        for imptarget in impact.split():
          if imptarget=="Shutdown" or imptarget=="Reboot":
            imptarget="Shutdown/Reboot"
          try:
              sim_to_sentence2 += modelimp.wv.similarity(word, imptarget)
          except KeyError:
              pass # ignore words that aren't in vocabulary
      if type(method)==str:
        for target in method.split():
            try:
                sim_to_sentence += modeloff.wv.similarity(word, target)
            except KeyError:
                pass # ignore words that aren't in vocabulary
      if type(impact)==float:
        logimpact=impact
      sim_to_sentence /= len(sentence_words)
      sim_to_sentence2 /= len(sentence_words)
    sentences_similarity[idx] += sim_to_sentence
    sentences_similarity2[idx] += sim_to_sentence2
  result = list(zip(sentences_similarity, offensivetech))
  result=heapq.nlargest(1, result, key=lambda x:x[0])

  result2 = list(zip(sentences_similarity2, offensivetech))
  result2=heapq.nlargest(1, result2, key=lambda x:x[0])
  jsonresult={'CVEID': listevul[vul][0],'ImpactMethod':listevul[vul][1],'LogicalImpact':logimpact, 'Context':listevul[vul][3], 'result': result}
  listresult.append(jsonresult)
  jsonresult2={'CVEID': listevul[vul][0],'ImpactMethod':listevul[vul][1],'LogicalImpact':logimpact, 'Context':listevul[vul][3], 'result': result2}
  listresult2.append(jsonresult2)

In [ ]:
print(listresult)

[{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'ImpactMethod': 'Code Execution', 'LogicalImpact': 'Shutdown', 'Context': 'Application', 'result': [(0.6636689007282257, 'Service Execution')]}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'ImpactMethod': 'Code Execution', 'LogicalImpact': 'Manipulation', 'Context': 'Application', 'result': [(0.6636689007282257, 'Service Execution')]}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'ImpactMethod': 'Code Execution', 'LogicalImpact': 'Discovery', 'Context': 'Application', 'result': [(0.6636689007282257, 'Service Execution')]}]


In [ ]:
for i in listresult:
  print(i)

{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'ImpactMethod': 'Code Execution', 'LogicalImpact': 'Shutdown', 'Context': 'Application', 'result': [(0.6636689007282257, 'Service Execution')]}
{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'ImpactMethod': 'Code Execution', 'LogicalImpact': 'Manipulation', 'Context': 'Application', 'result': [(0.6636689007282257, 'Service Execution')]}
{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'ImpactMethod': 'Code Execution', 'LogicalImpact': 'Discovery', 'Context': 'Application', 'result': [(0.6636689007282257, 'Service Execution')]}


In [ ]:
for e in listresult2:
  print(e)

{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'ImpactMethod': 'Code Execution', 'LogicalImpact': 'Shutdown', 'Context': 'Application', 'result': [(0.5, 'System Shutdown/Reboot')]}
{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'ImpactMethod': 'Code Execution', 'LogicalImpact': 'Manipulation', 'Context': 'Application', 'result': [(0.4999999701976776, 'Data Manipulation')]}
{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'ImpactMethod': 'Code Execution', 'LogicalImpact': 'Discovery', 'Context': 'Application', 'result': [(0.4999999701976776, 'Process Discovery')]}


In [ ]:
#Modifier pour que ça prenne l'un ou l'autre entre impact et méthode sans tenir compte du max, il suffit que le score soit suppérieur à 0.2
listresult4=[]
for res in listresult:
  for res2 in listresult2:
    if res['CVEID']==res2['CVEID']:
      #if  max(res['result'][0][0],res2['result'][0][0])>0.15:
      if res['result'][0][0]>0.1:
        #if max(res['result'][0][0],res2['result'][0][0])==res['result'][0][0]:
          #matching = [s for s in offens if res['result'][0][1] in s]
          #print(matching)
          result={'CVEID':res['CVEID'], 'Context':res['Context'], 'Method or Impact':res['ImpactMethod'],'OffensiveTechnique':res['result'][0][1]}
          if result not in listresult4:
            listresult4.append(result)
          #res['OffensiveTechnique'] = res['result'][0][1]
      if res2['result'][0][0]>0.1:
        #else:
          #matching = [s for s in offens if res2['result'][0][1] in s]
          #print(matching)
          result={'CVEID':res2['CVEID'], 'Context':res2['Context'], 'Method or Impact':res2['LogicalImpact'],'OffensiveTechnique':res2['result'][0][1]}
          if result not in listresult4:
            listresult4.append(result)
          #res['OffensiveTechnique'] = res2['result'][0][1]

In [ ]:
print(listresult4)

[{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Code Execution', 'OffensiveTechnique': 'Service Execution'}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Shutdown', 'OffensiveTechnique': 'System Shutdown/Reboot'}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Manipulation', 'OffensiveTechnique': 'Data Manipulation'}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Discovery', 'OffensiveTechnique': 'Process Discovery'}]


In [ ]:
listresult6=[]
for e in range(len(listresult4)):
  if listresult4[e]["CVEID"]==listresult4[e-1]["CVEID"] and listresult4[e]["OffensiveTechnique"]==listresult4[e-1]["OffensiveTechnique"]:
    listresult6.append(listresult4[e])
  """if e not in listresult3:
    cve=e["CVEID"]
    listresult3.append(e)"""
if len(listresult4)!=1:
  listresultfinal = [i for i in listresult4 if i not in listresult6]
else:
  listresultfinal=listresult6

In [ ]:
print(listresultfinal)

[{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Code Execution', 'OffensiveTechnique': 'Service Execution'}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Shutdown', 'OffensiveTechnique': 'System Shutdown/Reboot'}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Manipulation', 'OffensiveTechnique': 'Data Manipulation'}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Discovery', 'OffensiveTechnique': 'Process Discovery'}]


In [ ]:
print(listresult4)

[{'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Code Execution', 'OffensiveTechnique': 'Service Execution'}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Shutdown', 'OffensiveTechnique': 'System Shutdown/Reboot'}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Manipulation', 'OffensiveTechnique': 'Data Manipulation'}, {'CVEID': 'http://www.semanticweb.org/anonymous/ontologies/2022/6/untitled-ontology-4#CVECVE-2017-0144', 'Context': 'Application', 'Method or Impact': 'Discovery', 'OffensiveTechnique': 'Process Discovery'}]


In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/d3fend.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def searchar(t):
    artifact=[]
    sparql = """select Distinct ?l where{
          {
              ?sb rdfs:subClassOf ?of.
              ?of rdfs:subClassOf+ <http://d3fend.mitre.org/ontologies/d3fend.owl#OffensiveTechnique> .
              ?of <http://www.w3.org/2000/01/rdf-schema#label> '"""+t+"""' .
              ?sb ?p ?b .
              ?b rdfs:subClassOf* <http://d3fend.mitre.org/ontologies/d3fend.owl#Artifact> .
              ?b <http://www.w3.org/2000/01/rdf-schema#label> ?l .
          }
          UNION
          {
              ?of rdfs:subClassOf+ <http://d3fend.mitre.org/ontologies/d3fend.owl#OffensiveTechnique> .
              ?of <http://www.w3.org/2000/01/rdf-schema#label> '"""+t+"""' .
              ?of ?p ?b .
              ?b rdfs:subClassOf* <http://d3fend.mitre.org/ontologies/d3fend.owl#Artifact> .
              ?b <http://www.w3.org/2000/01/rdf-schema#label> ?l .
          }
        }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['l'].toPython())
        artifact.append(s)
    return artifact

In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/d3fend.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def searchard(t):
    artifact=[]
    sparql = """select Distinct ?l where{
                    ?of rdfs:subClassOf+ ?o .
                    ?o <http://www.w3.org/2000/01/rdf-schema#label> '""" +t+"""' .
                    ?of <http://www.w3.org/2000/01/rdf-schema#label> ?l .
                }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['l'].toPython())
        artifact.append(s)
    return artifact

In [ ]:
for r in range(len(listresultfinal)):
      artifacts=[]
      technique=str(listresultfinal[r]['OffensiveTechnique'])
      art=searchar(technique)
      if len(art)>0:
        for a in art:
          if a=='Digital Artifact' and len(art)==1:
            artifactss=searchard(a)
            modelart = gensim.models.Word2Vec.load("word2vecart.model")
            sentences_similarity3 = np.zeros(len(artifactss))
            for idx, sentence in enumerate(artifactss):
              context=listresultfinal[r]['Context']
              sentence_words3 = sentence.split()
              for word in sentence_words3:
                sim_to_sentence3 = 0
                for target in context.split():
                    try:
                        sim_to_sentence3 += modelart.wv.similarity(word, target)
                    except KeyError:
                        pass # ignore words that aren't in vocabulary
                sim_to_sentence3 /= len(sentence_words3)
              sentences_similarity3[idx] += sim_to_sentence3
              if len(set(sentence_words3).intersection(context.split()))>=len(context.split())-1:
                sentences_similarity3[idx]=sentences_similarity3[idx]+0.5
            result3 = list(zip(sentences_similarity3, artifactss))
            for z in result3:
              intersec=set(z[1].split()).intersection(context.split())
            result3=heapq.nlargest(5, result3, key=lambda x:x[0])
            artifacts.append(result3[0][1])
          else:
            if a!='Digital Artifact':
              artifacts.append(a)
      else:
        artifactss=searchard('Digital Artifact')
        #print(artifactss)
        modelart = gensim.models.Word2Vec.load("word2vecart.model")
        sentences_similarity3 = np.zeros(len(artifactss))
        for idx, sentence in enumerate(artifactss):
          context=listresultfinal[r]['Context']
          sentence_words3 = sentence.split()
          for word in sentence_words3:
            sim_to_sentence3 = 0
            #print(context)
            if type(context)==str:
              for target in context.split():
                  try:
                      sim_to_sentence3 += modelart.wv.similarity(word, target)
                  except KeyError:
                      pass # ignore words that aren't in vocabulary
              sim_to_sentence3 /= len(sentence_words3)
            sentences_similarity3[idx] += sim_to_sentence3
          if type(context)==str:
            if len(set(sentence_words3).intersection(context.split()))>=len(context.split())-1:
              sentences_similarity3[idx]=sentences_similarity3[idx]+0.5
        result3 = list(zip(sentences_similarity3, artifactss))
        for z in result3:
          if type(context)==str:
            intersec=set(z[1].split()).intersection(context.split())
        result3=heapq.nlargest(5, result3, key=lambda x:x[0])
        artifacts.append(result3[0][1])
      listresultfinal[r]['Artifact'] = artifacts

In [ ]:
modelart = gensim.models.Word2Vec.load("word2vecart.model")
for vul in range(len(listresultfinal)):
  sentences_similarity3 = np.zeros(len(listresultfinal[vul]['Artifact']))
  for idx, sentence in enumerate(listresultfinal[vul]['Artifact']):
    #print(sentences_similarity,sentence)
    context=listresultfinal[vul]['Context']
    #print(impact)
    #print(context,idx,sentence)
    sentence_words3 = sentence.split()
    #print(sentence_words)
    for word in sentence_words3:
      sim_to_sentence3 = 0
      #print(context)
      if type(context)==str:
        for target in context.split():
            #print(idx,target,word)
            if type(target)!=float:
              try:
                  sim_to_sentence3 += modelart.wv.similarity(word, target)
              except KeyError:
                  pass # ignore words that aren't in vocabulary
        sim_to_sentence3 /= len(sentence_words3)
    sentences_similarity3[idx] += sim_to_sentence3
  result3 = list(zip(sentences_similarity3, listresultfinal[vul]['Artifact']))
  #result3=heapq.nlargest(1, result3, key=lambda x:x[0])

In [ ]:
#Nouvelle approche
#class SparqlQueries:
#    def __init__(self):
my_world = World()
my_world.get_ontology("/content/d3fend.owl").load() #path to the owl file is given here
#sync_reasoner(my_world)  #reasoner is started and synchronized here
graph = my_world.as_rdflib_graph()

def search(asset):
    countermeasure=[]
    propriete=[]
    sparql = """select ?label ?def ?desc ?p where {
    <http://d3fend.mitre.org/ontologies/d3fend.owl#"""+asset+"""> rdfs:subClassOf* ?class .
        {
            select ?label ?def ?desc ?p where{
                    ?def rdfs:subClassOf+ <http://d3fend.mitre.org/ontologies/d3fend.owl#DefensiveTechnique> .
                    ?def ?p ?class .
                    ?def rdfs:label ?label .
                    ?def <http://d3fend.mitre.org/ontologies/d3fend.owl#definition> ?desc .
                }
        }
    }"""
    #query is being run
    resultsList = graph.query(sparql)
    for row in resultsList:
        s = str(row['label'].toPython())
        p = str(row['p'].toPython())
        countermeasure.append(s)
        propriete.append(s+","+p)
    return countermeasure, propriete

In [ ]:
from pandas.io.parsers import python_parser
#Nouvelle approche
removeart=[]
removecounter=[]
datasetcounter=[]
countermeasure=[]
propriete=[]
phase=""
category=""
eradicationverb=["http://d3fend.mitre.org/ontologies/d3fend.owl#deletes","http://d3fend.mitre.org/ontologies/d3fend.owl#evicts","http://d3fend.mitre.org/ontologies/d3fend.owl#terminates","http://d3fend.mitre.org/ontologies/d3fend.owl#may-evict"]
#identificationverb=["http://d3fend.mitre.org/ontologies/d3fend.owl#monitors","http://d3fend.mitre.org/ontologies/d3fend.owl#detects","http://d3fend.mitre.org/ontologies/d3fend.owl#analyzes", "http://d3fend.mitre.org/ontologies/d3fend.owl#verifies", "http://d3fend.mitre.org/ontologies/d3fend.owl#identifies", "http://d3fend.mitre.org/ontologies/d3fend.owl#evaluates"]
identificationverb=["http://d3fend.mitre.org/ontologies/d3fend.owl#evaluates","http://d3fend.mitre.org/ontologies/d3fend.owl#validates","http://d3fend.mitre.org/ontologies/d3fend.owl#monitors","http://d3fend.mitre.org/ontologies/d3fend.owl#detects","http://d3fend.mitre.org/ontologies/d3fend.owl#neutralizes", "http://d3fend.mitre.org/ontologies/d3fend.owl#verifies","http://d3fend.mitre.org/ontologies/d3fend.owl#analyzes","http://d3fend.mitre.org/ontologies/d3fend.owl#restricts","http://d3fend.mitre.org/ontologies/d3fend.owl#spoofs","http://d3fend.mitre.org/ontologies/d3fend.owl#deceives-with"]
containmentverb=["http://d3fend.mitre.org/ontologies/d3fend.owl#updates","http://d3fend.mitre.org/ontologies/d3fend.owl#suspends","http://d3fend.mitre.org/ontologies/d3fend.owl#may-contain","http://d3fend.mitre.org/ontologies/d3fend.owl#use-limits","http://d3fend.mitre.org/ontologies/d3fend.owl#obfuscates","http://d3fend.mitre.org/ontologies/d3fend.owl#disables","http://d3fend.mitre.org/ontologies/d3fend.owl#may-disable","http://d3fend.mitre.org/ontologies/d3fend.owl#use-limits","http://d3fend.mitre.org/ontologies/d3fend.owl#encrypts","http://d3fend.mitre.org/ontologies/d3fend.owl#limits","http://d3fend.mitre.org/ontologies/d3fend.owl#authenticates","http://d3fend.mitre.org/ontologies/d3fend.owl#filters","http://d3fend.mitre.org/ontologies/d3fend.owl#isolates","http://d3fend.mitre.org/ontologies/d3fend.owl#hardens","http://d3fend.mitre.org/ontologies/d3fend.owl#blocks"]
#containmentverb=["http://d3fend.mitre.org/ontologies/d3fend.owl#validates","http://d3fend.mitre.org/ontologies/d3fend.owl#may-contain","http://d3fend.mitre.org/ontologies/d3fend.owl#use-limits","http://d3fend.mitre.org/ontologies/d3fend.owl#encrypts","http://d3fend.mitre.org/ontologies/d3fend.owl#restricts","http://d3fend.mitre.org/ontologies/d3fend.owl#limits""http://d3fend.mitre.org/ontologies/d3fend.owl#authenticates","http://d3fend.mitre.org/ontologies/d3fend.owl#filters","http://d3fend.mitre.org/ontologies/d3fend.owl#isolates","http://d3fend.mitre.org/ontologies/d3fend.owl#hardens","http://d3fend.mitre.org/ontologies/d3fend.owl#blocks","http://d3fend.mitre.org/ontologies/d3fend.owl#spoofs","http://d3fend.mitre.org/ontologies/d3fend.owl#deceives-with"]
recoveryverb=["http://d3fend.mitre.org/ontologies/d3fend.owl#strengthens","http://d3fend.mitre.org/ontologies/d3fend.owl#enables","http://d3fend.mitre.org/ontologies/d3fend.owl#regenerates","http://d3fend.mitre.org/ontologies/d3fend.owl#restores"]
preparationverb=["http://d3fend.mitre.org/ontologies/d3fend.owl#may-access","http://d3fend.mitre.org/ontologies/d3fend.owl#manages"]
for result in listresultfinal:
    for res in result['Artifact']:
        request=search(res.replace(" ", ""))
        countermeasure=[i for n, i in enumerate(request[0]) if i not in request[0][:n]]
        propriete=[i for n, i in enumerate(request[1]) if i not in request[1][:n]]
        #print(res,len(propriete),len(countermeasure))
        for count in range(len(countermeasure)):
            if res.find("File") >= 0 or res.find("Script") >= 0:
                category="File"
            else:
              if res.find("Process") >= 0:
                category="Process"
              else:
                if res.find("Email") >= 0:
                  category="Email"
                else:
                  if res.find("Network") >= 0 or res.find("Address") >= 0 or res.find("IP") >= 0 or res.find("Domain") >= 0 or res.find("URL") >= 0 or res.find("Domain") >= 0 or res.find("Log") >= 0:
                    category="Network"
                  else:
                    if res.find("User") >= 0 or res.find("Token") >= 0 or res.find("Credential") >= 0 or res.find("Session") >= 0:
                      category="Identity"
                    else:
                      if res.find("Service") >= 0 or res.find("Key") >= 0:
                        category="Configuration"
                      else:
                        category="General"
            #print(res,index)
            #print(vdo.search_one(iri = "*"+result["CVEID"].split("#")[1]).hasIdentity[0].value[0])
            cveid=vdo.search_one(iri = "*"+result["CVEID"].split("#")[1]).hasIdentity.value[0]
            #print(vdo.search_one(iri = "*"+result["CVEID"].split("#")[1]).hasIdentity.value[0])
            if propriete[count].split(',')[1] in eradicationverb:
              phase="Eradication"
              datasetcounter.append({'CVEID':cveid, 'Method or Impact':result['Method or Impact'], 'Context':result['Context'], 'Artifact':res, 'Countermeasure':countermeasure[count], 'Phase':phase, 'Category':category})
              phase=""
            else:
              if propriete[count].split(',')[1] in identificationverb:
                phase="Identification"
                #cveid=vdo.search_one(iri = "*"+result["CVEID"].split("#")[1]).hasIdentity[0].value[0]
                datasetcounter.append({'CVEID':cveid, 'Method or Impact':result['Method or Impact'], 'Context':result['Context'], 'Context':result['Context'], 'Artifact':res, 'Countermeasure':countermeasure[count], 'Phase':phase, 'Category':category})
                phase=""
              else:
                if propriete[count].split(',')[1] in containmentverb:
                  phase="Containment"
                  #cveid=vdo.search_one(iri = "*"+result["CVEID"].split("#")[1]).hasIdentity[0].value[0]
                  datasetcounter.append({'CVEID':cveid, 'Method or Impact':result['Method or Impact'], 'Context':result['Context'], 'Context':result['Context'], 'Artifact':res, 'Countermeasure':countermeasure[count], 'Phase':phase, 'Category':category})
                  phase=""
                else:
                  if propriete[count].split(',')[1] in recoveryverb:
                    phase="Recovery"
                    #cveid=vdo.search_one(iri = "*"+result["CVEID"].split("#")[1]).hasIdentity[0].value[0]
                    datasetcounter.append({'CVEID':cveid, 'Method or Impact':result['Method or Impact'], 'Context':result['Context'], 'Context':result['Context'], 'Artifact':res, 'Countermeasure':countermeasure[count], 'Phase':phase, 'Category':category})
                    phase=""
                  else:
                    if propriete[count].split(',')[1] in preparationverb:
                      phase="Preparation"
                      #cveid=vdo.search_one(iri = "*"+result["CVEID"].split("#")[1]).hasIdentity[0].value[0]
                      datasetcounter.append({'CVEID':cveid, 'Method or Impact':result['Method or Impact'], 'Context':result['Context'], 'Context':result['Context'], 'Artifact':res, 'Countermeasure':countermeasure[count], 'Phase':phase, 'Category':category})
                      phase=""

In [ ]:
#output
#Nouvelle approche
tsv_file = open("/content/countermeasureexec.tsv", "w")
tsv_writer = csv.writer(tsv_file, delimiter='\t')

tsv_writer.writerow(datasetcounter[0].keys()) # write the header

for row in datasetcounter: # write data rows
    tsv_writer.writerow(row.values())

tsv_file.close()

In [ ]:

import pandas as pd
df = pd.read_csv("countermeasureexec.tsv",sep='\t')
df.tail(50)

,CVEID,Method or Impact,Context,Artifact,Countermeasure,Phase,Category
0,CVE-2017-0144,Code Execution,Application,Application,Asset Vulnerability Enumeration,Identification,General
1,CVE-2017-0144,Code Execution,Application,Application,Software Update,Containment,General
2,CVE-2017-0144,Code Execution,Application,Application,Restore Software,Recovery,General
3,CVE-2017-0144,Shutdown,Application,Application,Asset Vulnerability Enumeration,Identification,General
4,CVE-2017-0144,Shutdown,Application,Application,Software Update,Containment,General
5,CVE-2017-0144,Shutdown,Application,Application,Restore Software,Recovery,General
6,CVE-2017-0144,Manipulation,Application,File,File Encryption,Containment,File
7,CVE-2017-0144,Manipulation,Application,File,Local File Permissions,Identification,File
8,CVE-2017-0144,Manipulation,Application,File,Decoy File,Identification,File
9,CVE-2017-0144,Manipulation,Application,File,File Analysis,Identification,File
